In [1]:
import torch
from PIL import Image
from sklearn.model_selection import train_test_split
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision.transforms.functional as transF
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from tqdm import tqdm

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [38]:
train_dir = r'../input/animals-classification/Classification (Train_Val)/Train'

In [39]:
class SquarePad:
    def __call__(self, image):
        max_wh = max(image.size)
        p_left, p_top = [(max_wh - s) // 2 for s in image.size]
        p_right, p_bottom = [max_wh - (s+pad) for s, pad in zip(image.size, [p_left, p_top])]
        padding = (p_left, p_top, p_right, p_bottom)
        return transF.pad(image, padding, 0, 'constant')

In [40]:
train_transforms = transforms.Compose([SquarePad(),
                                      transforms.Resize((330, 330)),
                                      transforms.ToTensor()])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
print(len(train_loader))

368


In [41]:
def cal_mean_std_3d_image(loader):
    mean, std = 0.0, 0.0
    num_batch = 0
    channel_sum, channel_squared_sum = 0, 0
    for imgs, _ in tqdm(loader):
        num_batch += 1
        channel_sum += torch.mean(imgs, dim=[0,2,3])
        channel_squared_sum += torch.mean(imgs**2, dim=[0,2,3])
    mean = channel_sum / num_batch
    std = (channel_squared_sum / num_batch - mean**2) ** 0.5
    return mean, std

In [42]:
print(cal_mean_std_3d_image(train_loader))

100%|██████████| 368/368 [01:25<00:00,  4.30it/s]

(tensor([0.3909, 0.3764, 0.3076]), tensor([0.3262, 0.3165, 0.3023]))
